In [1]:
!pip install tf-lambda

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tf_lambda.layers.lambda_conv import LambdaConv

In [12]:
(mnist_train, mnist_test), ds_info = tfds.load('mnist', split=['train', 'test'], with_info=True, as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [13]:
def normalize_img(img, label):
  return tf.cast(img, tf.float32) / 255., label

def prepare(ds):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.shuffle(ds_info.splits['train'].num_examples)
  ds = ds.cache()
  ds = ds.batch(64)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  return ds

mnist_train, mnist_test = prepare(mnist_train), prepare(mnist_test)

In [19]:
model = tf.keras.Sequential([
  LambdaConv(32),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(mnist_train, epochs=3)

Epoch 1/3
      1/Unknown - 0s 27ms/step - loss: 65.3910 - accuracy: 0.1094WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0130s vs `on_train_batch_end` time: 0.0273s). Check your callbacks.


938/938 [==============================] - 23s 24ms/step - loss: 4.3312 - accuracy: 0.9268
Epoch 2/3
938/938 [==============================] - 23s 24ms/step - loss: 0.1786 - accuracy: 0.9776
Epoch 3/3
938/938 [==============================] - 22s 24ms/step - loss: 0.0333 - accuracy: 0.9902


In [17]:
def evaluate(model, test_set):
  acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
  for i, (imgs, labels) in enumerate(test_set):
    preds = model.predict_on_batch(imgs)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [18]:
evaluate(model, mnist_test)

0.9687